# MTH3302 : Méthodes probabilistes et statistiques pour l'I.A.

Jonathan Jalbert<br/>
Professeur adjoint au Département de mathématiques et de génie industriel<br/>
Polytechnique Montréal<br/>


# Chapitre 5 : Modèles bayésiens pour la loi normale

Dans ce chapitre, nous utiliserons le jeu de données normaldata disponible sur le site web du cours. Ces données proviennent de l'expérience de Michelson-Morley effectuée par Illingworth en 1927 dans le but de mesurer la différence de la vitesse de la lumière dans deux directions orthogonales, dont l'une des directions est parallèle à la vitesse de l'éther et l'autre est orthogonale. La différence de vitesse a été mesurée par interférométrie en calculant le déplacement des franges d'interférence. La première colonne indique le temps de la journée où les essais se sont déroulés et la deuxième colonne correspond au déplacement moyen des franges d'interférence pour 10 essais indépendants.

Pour le montage expérimental de Illingworth en 1927, ce-dernier a estimé que l'erreur de mesure de son montage  correspondait à un écart-type de $0.75$ frange de déplacement. Il supposa que chacune des $n=64$ mesures étaient distribuées selon la loi normale $ Y_i \sim \mathcal{N}(\mu,0.75^2)$ où le paramètre $\mu$ inconnu correspond au vrai déplacement des franges d'interférence. Le nombre moyen du déplacement des franges d'interférence pour les 64 essais est de $\bar{y} = -0.015$.

Installation et chargement des librairies nécessaires.

In [ ]:
# using Pkg
# Pkg.add("SpecialFunctions")

In [ ]:
# Chargement des librairies
using CSV, DataFrames             # Pour charger et organiser les données
using Gadfly                      # Pour générer des graphiques
using LinearAlgebra               # Pour utiliser les fonctions d'algèbre linéaire
using Statistics                  # Libriairie contenant des fonctions statistiques de base
using SpecialFunctions            # Librairie contenant notamment la fonction Gamma
using Distributions               # Librairie contenant les lois statistiques

## 5.0 Chargement des données et analyse exploratoire

In [ ]:
# Chargement des observations effectuées par Illingworth en 1927 sur l'expérience de Michelson-Morley
data = CSV.read("normaldata.csv")
first(data,5)

In [ ]:
# Calcul des statistiques descriptives du jeu de données

n = length(data[:,:FringeDispl])
ȳ = mean(data[:,:FringeDispl])
s² = var(data[:,:FringeDispl])

println("Il y a $n observations.")
println("La moyenne des $n observations est de $ȳ.")
println("La variance des $n observations est de $(s²).")

In [ ]:
# Afficher les observations mesurées par Illingworth en fonction du moment de la journée

plot(data, x=:TimeOfDay, y=:FringeDispl, Geom.point)

In [ ]:
# Afficher l'histogramme des déplacements des franges d'interférence

nbin = round(sqrt(n))

plot(data, x=:FringeDispl, Geom.histogram(bincount=nbin))

## 5.1 Modèle gaussien

On estime le paramètre inconnu $\mu$ de la loi normale par la méthode du maximum de la vraisemblance. On peut par la suite calculer un intervalle de confiance de niveau 95% pour $\mu$. Rappelons que l'on suppose que la variance est connue et est égale à $\sigma^2 = 0.75^2$.

In [ ]:
# Estimation ponctuelle
println("μ̂ = ",ȳ)

# Estimation par intervalle de confiance
a = ȳ - 1.96*0.75
b = ȳ + 1.96*0.75
println("Intervalle de confiance à 95% : [$a,$b]")


## 5.2 Estimation bayésienne de $\mu$ lorsque la variance est connue

On estime le paramètre inconnu $\mu$ de la loi normale par la méthode bayésienne. On suppose que la variance est connue et est égale à $\sigma^2 = 0.75^2$.

### 5.2.2 Loi a priori normale

Pour la vraisemblance normale avec la variance connue, la loi *a priori* conjuguée pour la moyenne $\mu$ est la loi normale. Utilisons la loi moyenne nulle et de variance $\sigma^2 = 0.75^2$ comme loi a priori. La loi *a posteriori* est donc aussi gaussienne.

In [ ]:
# la variance est connue
σ = 0.75

# définition de la loi a priori
prior = Normal(0,σ)

# Stockage des densités dans un DataFrame pour un affichage rapide
x = range(-1,stop=1,length=501)
df = DataFrame(μ = x)
df[!,Symbol("n=0")] = pdf.(prior,x)

# calcul de la loi a posteriori pour n=5 observations
n = 5
ȳ = mean(data[1:n,:FringeDispl])
posterior = Normal(n/(n+1)*ȳ,σ/sqrt(n+1))
df[!,Symbol("n=5")] = pdf.(posterior,x)

# calcul de la loi a posteriori pour les n=64 observations
n = length(data[:,:FringeDispl])
ȳ = mean(data[1:n,:FringeDispl])
posterior = Normal(n/(n+1)*ȳ,σ/sqrt(n+1))
df[!,Symbol("n=64")] = pdf.(posterior,x)

df = melt(df, :μ)
rename!(df, :value => :densité, :variable => :observations)

plot(df,x=:μ, y=:densité, color=:observations, Geom.line)


## 5.2.4 Loi a priori non informative

Utilisons la loi *a priori* non-informative impropre suivante pour $\mu$ :
$$f_\mu(\mu) \propto 1.$$

In [ ]:
# la variance est connue
σ = 0.75

# définition de la loi a priori
prior = Normal(0,σ)

# Stockage des densités dans un DataFrame pour un affichage rapide
x = range(-1,stop=1,length=501)
df = DataFrame(μ = x)
df[!,Symbol("n=0")] = pdf.(prior,x)

# calcul de la loi a posteriori pour n=5 observations
n = 5
ȳ = mean(data[1:n,:FringeDispl])
posterior = Normal(ȳ,σ/sqrt(n))
df[!,Symbol("n=5")] = pdf.(posterior,x)

# calcul de la loi a posteriori pour n=564 observations
n = length(data[:,:FringeDispl])
ȳ = mean(data[1:n,:FringeDispl])
posterior = Normal(ȳ,σ/sqrt(n))
df[!,Symbol("n=64")] = pdf.(posterior,x)

df = melt(df, :μ)
rename!(df, :value => :densité, :variable => :observations)

plot(df,x=:μ, y=:densité, color=:observations, Geom.line)


Comparaison des lois *a posteriori* en fonction du caractère informatif ou non-informatif de la loi *a priori*.

In [ ]:

# Calcul de la loi a posteriori
n = 5
ȳ = mean(data[1:n,:FringeDispl])
posterior_info = Normal(n/(n+1)*ȳ,σ/sqrt(n+1))
posterior_noninfo = Normal(ȳ,σ/sqrt(n))

# Affichage de la loi a priori et de la loi a posteriori
x = range(-1,stop=1,length=101)

infoLayer = layer(x=x, y=pdf.(posterior_info,x), Geom.line, Theme(default_color="deepskyblue"))
noninfoLayer = layer(x=x, y=pdf.(posterior_noninfo,x), Geom.line, Theme(default_color="red"))
fig1 = plot(infoLayer, noninfoLayer, Guide.xlabel("μ"), Guide.title("n=5"),
    Guide.manual_color_key("Légende", ["Informatif", "Non informatif"], ["deepskyblue","red"]))


ȳ = mean(data[:,:FringeDispl])
n = size(data,1)
posterior_info = Normal(n/(n+1)*ȳ,σ/sqrt(n+1))
posterior_noninfo = Normal(ȳ,σ/sqrt(n))

# Affichage de la loi a priori et de la loi a posteriori
x = range(-.5,stop=.5,length=101)

infoLayer = layer(x=x, y=pdf.(posterior_info,x), Geom.line, Theme(default_color="deepskyblue"))
noninfoLayer = layer(x=x, y=pdf.(posterior_noninfo,x), Geom.line, Theme(default_color="red"))
fig2 = plot(infoLayer, noninfoLayer, Guide.xlabel("μ"), Guide.title("n=64"),
    Guide.manual_color_key("Légende", ["Informatif", "Non informatif"], ["deepskyblue","red"]))

set_default_plot_size(10inch, 4inch)
gridstack([fig1 fig2])


### 5.2.5 Estimation bayésienne ponctuelle

Utilisons la loi *a priori* non-informative. Puisque la loi *a posteriori* est symétrique, la moyenne et le mode concordent.

In [ ]:
# Estimation ponctuelle
μ̂ = ȳ
println("μ̂ = ",μ̂)

# Une autre façon d'obtenir l'estimation aurait été à l'aide de la fonction mode ou mean de la librairie Distributions.
# posterior = Normal(ȳ,σ/sqrt(n))
# μ̂ = mean(posterior)
# μ̂ = mode(posterior)

### 5.2.6 Estimation par intervalle de crédibilité bayésien

Utilisons la loi *a priori* non-informative. Puisque la loi *a posteriori* est symétrique, l'intervalle de crédibilité usuel et l'intervalle de plus haute densité *a posteriori* concordent.

In [ ]:
posterior = Normal(ȳ,σ/sqrt(n))

I = quantile.(posterior,[.025 .975])

println("L'intervalle de crédibilité bayésien de niveau 95% pour μ est [$(I[1]) , $(I[2])].")

### 5.2.7 Distribution prédictive

Utilisons la loi *a priori* non-informative.

In [ ]:
predictive = Normal(ȳ,sqrt( (n+1)/n )*σ )

x = range(-.5,stop=.5,length=501)
df = DataFrame(ỹ = x)
df[!,:f] = pdf.(posterior,x)

set_default_plot_size(5inch, 4inch)
plot(df, x=:ỹ, y=:f, Geom.line)

## 5.2.3 Loi a priori informative non conjuguée

Utilisons maintenant la loi de Student centrée réduite à 5 degrés de libertés comme loi a priori. Dans ce cas, la loi a posteriori ne s'exprime pas sous une forme analytique. Il faudra recourir à l'algorithme de Metropolis-Hastings pour générer une échantillon de la loi a posteriori et estimer la moyenne et l'intervalle de crédibilité.

Dans un premier temps, la densité (ou plutôt la log densité) de la loi de Student à $\nu$ degrés de liberté et avec le paramètre de localisation $\mu$ et le paramètre d'échelle $\sigma$ doit être implantée dans Julia. C'est ce que fait la fonction suivante.

In [ ]:
# Implémentation de la densité Student à ν degrés de liberté avec paramètres de localisation μ et d'échelle σ
function Studentpdf(y::Real,ν::Int,μ::Real,σ::Real)
    
    @assert σ > 0 "The scale parameter must be positive"
    
    z = (y-μ)/σ
    
    density = gamma( (ν+1)/2 ) / gamma(ν/2) / sqrt(ν*π) / sqrt(σ) * (1 + 1/ν * z^2 )^(-ν/2-1/2)
    
    return density
    
end

# Implémentation du log de la densité Student à ν degrés de liberté avec paramètres de localisation μ et d'échelle σ
function Studentlogpdf(y::Float64,ν::Int64,μ::Float64,σ::Float64)
    
    @assert σ > 0 "The scale parameter must be positive"
    
    z = (y-μ)/σ
    
    logdensity = lgamma( (ν+1)/2 ) - lgamma(ν/2) - 1/2*log(ν*π) - 1/2*log(σ)  - 1/2*(ν+1)*log(1 + (z^2)/ν )
    
    return logdensity
    
end

Pour mettre en oeuvre l'algorithme de Metropolis-Hastings, on doit évaluer le log de la vraisemblance plus le log de la loi a priori. La fonction suivante permet de simplifier l'écriture de cette évaluation.

In [ ]:
function logf(y::Array{Float64},μ::Float64)
    
    ldens = sum( logpdf.(Normal(μ,σ),y) ) + Studentlogpdf(μ,5,0.0,1.0)
    
end

In [ ]:

NITER = 1000

μ = zeros(NITER)
acc_cand = falses(NITER)

y = data[:,:FringeDispl]
n = length(y)

# état initial de la chaîne
μ[1] = -1

for j=2:NITER

    # proposition du candidat
    cand = μ[j-1] + rand(Normal(0,.15))
    
    # évaluation du ln ρ 
    lr = logf(y,cand) - logf(y,μ[j-1])
    
    # génération d'une valeur de la loi uniforme sur l'intervalle (0,1)
    u = rand(Uniform(0,1))
    
    # acceptation ou refus du candidat
    if lr > log(u)
        μ[j] = cand
        acc_cand[j] = true 
    else
        μ[j] = μ[j-1]
    end
    
end

pct = count(acc_cand)/NITER

println("Pourcentage d'acceptation des candidats = $pct")

MCMC = DataFrame(iter = collect(1:NITER), μ = μ)

plot(MCMC, x=:iter, y=:μ, Geom.line)


### Affichage de la loi a posteriori estimée

In [ ]:
# Afficher l'histogramme des données
WARMUP = 100
MCMC = MCMC[WARMUP+1:end,:]
m = size(MCMC,1)

set_default_plot_size(14cm, 8cm)
plot(MCMC, x=:μ, Geom.histogram(bincount=round(sqrt(m))))

In [ ]:
# Estimation ponctuelle
x = MCMC[:,:μ]
μ̂ = mean(x)
println("μ̂ = ",μ̂)

# Estimation par intervalle de confiance
a = quantile(x,.025)
b = quantile(x,.975)
println("Intervalle de confiance à 95% : [$a,$b]")

## 5.3 Estimation bayésienne de $\mu$ et $\sigma^2$

On estime les paramètre inconnus $\mu$ et $\sigma^2$ de la loi normale par la méthode bayésienne.

## 5.3.2 Loi a priori non informative

Si on prend la loi a priori impropre non informative 
$$f_{(\mu,\sigma^2)}(\mu,\sigma^2) \propto \frac{1}{\sigma^2},$$
alors la loi a posteriori ne s'exprime pas sous une forme connue. Il n'y a que les lois conditionnelles complètes complètes $f_{(\mu|Y=y,\sigma^2)}(\mu)$ et $f_{(\sigma^2|Y=y,\mu)}(\sigma^2)$ qui s'expriment sous une forme connue.


In [ ]:
# Loi conditionnelle complète de μ et σ²

y = data[:,:FringeDispl]
n = length(y)

ȳ = mean(y)
s² = var(y)

μ_ccd(σ²) = Normal(ȳ,sqrt(σ²/n))
σ²_ccd(μ) = InverseGamma(n/2, 1/2*sum( (y[i]-μ)^2 for i=1:n ))

### Échantillonnage de Gibbs

Les deux lois conditionnelles complètes permettent d'implémenter l'échantillonnage de Gibbs pour générer un échantillon de la loi a posteriori.

In [ ]:
NITER = 1000
μ = Array{Float64}(undef, NITER)
σ² = Array{Float64}(undef, NITER)

μ[1] = -.2
σ²[1] = .15

for i=2:NITER
   
    μ[i] = rand(μ_ccd(σ²[i-1]))
    
    σ²[i] = rand(σ²_ccd(μ[i]))
    
end

# Stockage des résulats dans un dataframe
MCMC = DataFrame(Iter = 1:NITER, μ = μ, σ² = σ²);

In [ ]:
set_default_plot_size(10inch, 4inch)
fig1 = plot(MCMC, x=:Iter, y=:μ, Geom.line)
fig2 = plot(MCMC, x=:Iter, y=:σ², Geom.line)
hstack(fig1,fig2)

In [ ]:
# Retirer les itérations correspondantes à la phase de chauffe de l'algorithme
WARMUP = 100
ind = MCMC[:,:Iter] .<= 100
deleterows!(MCMC, ind)
m = size(MCMC,1)

# traçage des densités marginales a posteriori
fig1 = plot(MCMC,x=:μ, Geom.histogram(bincount=round(sqrt(m))))
fig2 = plot(MCMC,x=:σ², Geom.histogram(bincount=round(sqrt(m))))
hstack(fig1,fig2)

### Calcul d'un estimateur ponctuel de $\mu$ et d'un intervalle de crédibilité à 95%

Vous remarquerez que l'intervalle de crédibilité pour $\mu$ est beaucoup plus court que lorsque l'on supposait la variance connue.

In [ ]:
# Estimation ponctuelle
x = MCMC[:,:μ]
μ̂ = mean(x)
println("μ̂ = ",μ̂)

# Estimation par intervalle de confiance
a = quantile(x,.025)
b = quantile(x,.975)
println("Intervalle de confiance à 95% : [$a,$b]")

### Calcul d'un estimateur ponctuel de $\sigma$ et d'un intervalle de crédibilité à 95%

Ici, on souhaite obtenir une estimation de $\sigma$ et non de $\sigma^2$.

On remarquera que l'erreur expérimentale associée au montage de Illingworth correspond à environ 0.20 franges d'interférence de déplacement. Illingworth a donc largement sous-estimé la précision de son montage.

In [ ]:
# Estimation ponctuelle
x = MCMC[:,:σ²]
x = sqrt.(x)
σ̂ = mean(x)
println("σ̂ = ", σ̂)

# Estimation par intervalle de confiance
a = quantile(x,.025)
b = quantile(x,.975)
println("Intervalle de confiance à 95% : [$a,$b]")

## 5.3.3 Densité marginale a posteriori de $\mu$.

Dans le cas de la vraisemblance normale avec une loi a priori partiellement conjuguée ou non informative, on peut obtenir une expression analytique pour la loi de $\mu$ a posteriori en intégrant $\sigma^2$ de la loi a posteriori. La loi de correspondant à l'histogramme de $\mu$ présenté précédemment est une loi de Student.


In [ ]:
s² = var(y)

xgrid = range(-.15, stop=.15,length=101)

μ_marg = DataFrame(μ=xgrid, density=Studentpdf.(xgrid,n-1,ȳ,sqrt(s²/n))) 

plot(μ_marg, x=:μ, y=:density, Geom.line )

## 5.4 Sélection de modèle

Dans le cas de l'expérience de Illingworth, on a deux modèles possibles :
- le modèle $M_1$ supposant la variance connue et égale à $0.75^2$ ;
- le modèle $M_2$ supposant la variance inconnue.

### Calculons le BIC du premier modèle

On utilisera ici les estimateurs du maximum de la vraisemblance par simplicité.

In [ ]:
y = data[:,:FringeDispl]
ȳ = mean(y)

μ̂ = ȳ
σ = 0.75 # variance supposée connue
n = size(data,1)

k = 1 # nombre de paramètres inconnus

BIC₁ = sum( logpdf.(Normal(μ̂, σ), y) ) - k/2*log(n)

### Calculons le BIC du deuxième modèle

On utilisera ici les estimateurs du maximum de la vraisemblance par simplicité.

In [ ]:
y = data[:,:FringeDispl]
ȳ = mean(y)

μ̂ = ȳ
σ̂² = var(y) 
n = size(data,1)

k = 2 # nombre de paramètres inconnus

BIC₂ = sum( logpdf.(Normal(μ̂, sqrt(σ̂²)), y) ) - k/2*log(n)

### Estimons le log du facteur de Bayes $B_{21}$ entre les deux modèles

Puisque $\ln B_{21} > 2$, la certitude que le modèle $M_1$ est faux par rapport au modèle $M_2$ est **décisive**.

In [ ]:
log_B21 = BIC₂ - BIC₁ 